# Truecolor_4: serving composite imagery using appengine

In the last colab, [Truecolor_3](https://colab.research.google.com/drive/1X_cqInqx_QjelsScxWnM53PxMhB-5mM4), we generated full disk composite images.

Let's save these images onto cloud storage, and create a web server to view them.

## Generating cloud images

We need to generate a large number of cloud images, 144 per day.  The easiest way to automate this is to set up a beam pipeline to process each of the satellite images, apply the model to predict cloud cover, and then save the result to cloud storage.

The code for this is in the [goes_truecolor/beam/make_cloud_masks.py](https://github.com/jyh/goes_truecolor/blob/master/goes_truecolor/beam/make_cloud_masks.py) file.

First, we get a set of files to process, collected by time

```
    reader = goes_reader.GoesReader(
        project_id=FLAGS.project,
        goes_bucket_name=FLAGS.goes_bucket,
        shape=(FLAGS.image_size, FLAGS.image_size))
    files = reader.list_time_range(start_date, end_date)
```

Then, the `CreateCloudMasks.process` method is called to produce the cloud mask from the satellite images at that time.  The first step is to read the satellite images.

```
    ir = self.reader.load_channel_images_from_files(file_table, self.ir_channels)
    ir_img, md = goes_reader.flatten_channel_images(ir, self.ir_channels)
```

Apply the model to produce the cloud mask.

```
    cloud_img = self.model.predict(ir_img)
```

Then produce a JPEG image and save it to cloud storage in a file with name format YYYY/DOY/YYYYMMDD_HHMMSS_US.jpg, where DOY is the day of year, and US is the microsecond timestamp.

```
    # Get jpeg bytes.
    cloud_img = Image.fromarray(cloud_img)
    buf = io.BytesIO()
    cloud_img.save(buf, format='JPEG', quality=70)
    buf = buf.getvalue()

    # Write to a file.
    t = md['time_coverage_start']
    filename = os.path.join(self.output_dir, t.strftime('%Y/%j/%Y%m%d_%H%M_%f.jpg'))
    logging.info('writing to %s', filename)
    blob = bucket.blob(filename)
    blob.upload_from_string(buf)
    logging.info('wrote to %s', filename)
```

You can run the pipeline using the following command.

```
$ make make_cloud_masks_dataflow
```

This will take a fair amount of time to run.  It cost me about $50 to create the images for the entire year 2019.

## Creating a web server

Google cloud also supports a product called [AppEngine](https://cloud.google.com/appengine) that can be used to create web servers.  There is a base quota that is free.

An AppEngine server can be written in many languages.  We'll use python 3.7, with the [Flask web microframework](https://www.fullstackpython.com/flask.html).  Flask uses function decorators to specify web paths to serve, and how to serve them.   

You can see the full code in [appengine/main.py](https://github.com/jyh/goes_truecolor/blob/master/appengine/main.py).  The core concept is in the cloud_masks function

```
@app.route('/cloud_masks/<int:year>/<int:day>/<string:filename>.jpg')
def handle_jpeg(year: int, day: int, filename: Text) -> flask.Response:
  name = urllib.parse.unquote(flask.request.path)
  manager = site_manager.SiteManager(name)
  img = manager.cloud_mask_jpeg()
  response = flask.make_response(img)
  response.headers.set('Content-Type', 'image/jpeg')
  return response  
```

The `@app.route` decorator specifies the web path to serve.  The cloud mask itself is constructed by the `SiteManager.cloud_mask_jpeg` method in [appengine/site_manager.py](https://github.com/jyh/goes_truecolor/blob/master/appengine/site_manager.py) that 1) fetches the world image, 2) fetchs the cloud mask from Google Cloud Storage, 3) combines the images, and 4) converts the result to a JPEG file.

```
  def cloud_mask_jpeg(self) -> bytes:
    """Return the current image in JPEG format."""
    # Get the world map.
    world_img = self.world_img()
    world_img = np.array(world_img, dtype=np.float32) / 256

    # Get the mask file.
    bucket = self.client.bucket(self.bucket_name)
    blob = bucket.blob(self.page_name[1:])
    s = blob.download_as_string()
    f = io.BytesIO(s)
    lum = Image.open(f)
    lum = np.array(lum)
    lum = lum.astype(np.float32) / 256
    lum = lum[:, :, np.newaxis]

    # Generate the composite.
    mask = 1 / (1 + np.exp(-10 * (lum - 0.3)))
    img = lum * mask + (1 - mask) * world_img
    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img)

    # Convert to JPEG.
    f = io.BytesIO()
    img.save(f, 'JPEG')
    return f.getvalue()
```

To deploy, you use the `gcloud` command.  In my case, I've set up an AppEngine project named `weather-324`.

```
$ gcloud app deploy --project=weather-324 app.yaml
```

<img src="https://docs.google.com/uc?export=download&id=1Rw-asL20nTCkpaSiDxZyJOIByTivhd6J">

You can access the server at http://weather-324.appspot.com.
## Finishing up

You can see from the image above that there are artifacts, like a brilliant white hole in the clouds over Canada.  Still, the results are fairly impressive for a simple machine learning system.